The specific piece of code in csdllib that does this is 
https://github.com/noaa-ocs-modeling/csdllib/blob/master/csdllib/data/coops.py -- the getData function (see below).

A sample URL that it downloads is https://opendap.co-ops.nos.noaa.gov/axis/webservices/waterlevelrawsixmin/plain/response.jsp?stationId=8410140&beginDate=20230302%2018:06&endDate=20230310%2012:00&datum=MSL&unit=0&timeZone=0&Submit=Submit

```
8410140 1 WL 2023-03-02 18:06  -1.868   0.017   1 0 0 0
8410140 1 WL 2023-03-02 18:12  -1.887   0.007   1 0 0 0
8410140 1 WL 2023-03-02 18:18  -1.901   0.010   0 0 0 0
8410140 1 WL 2023-03-02 18:24  -1.905   0.008   1 0 0 0
8410140 1 WL 2023-03-02 18:30  -1.910   0.010   1 0 0 0
8410140 1 WL 2023-03-02 18:36  -1.914   0.009   1 0 0 0
8410140 1 WL 2023-03-02 18:42  -1.923   0.009   0 0 0 0
```
Example code:
```
def getData (stationID,  dateRange, tmpDir=None,
             product='waterlevelrawsixmin', datum='MSL', units='meters', verbose=False):
    '''
    Allows for downloading the observations from NOAA's 
    Center for Operational Oceanographic Products and Services (COOPS)
    via OpenDAP server    http://opendap.co-ops.nos.noaa.gov .
    
    Args:
        stationID (str):              7 character-long CO-OPS station ID.
        dateRange (datetime, datetime): start and end dates of retrieval.
    '''

```

In [ ]:
from datetime import datetime

from searvey import coops

In [ ]:
st = coops.COOPS_Station(8410140)
ds = st.product(
        product='water_level',
        start_date=datetime(2023, 3, 2),
        end_date=datetime(2023, 3, 10),
        interval=None,  # Not applicable for waterlevel
        datum='MSL',
    )
ds

In [ ]:
df = ds.to_dataframe().drop(columns=['q', 'nws_id', 'x', 'y'])
df['f'] = df.f.str.split(',', expand=False)
#df[['f1', 'f2', 'f3', 'f4']] = df.f.str.split(',', expand=True)
#df = df.drop(columns=['f'])
df

In [ ]:
#f'{coopdid} 1 WL {yyyy_mm_dd} {hh_mm} {v: <7} {s: <7} {f: <9}'
out_string =(
    df.reset_index().to_string(
        formatters={
            'nos_id': '{0: <7} 1 WL'.format,
            't': lambda d: d.strftime('%Y-%m-%d %H:%M'),
            'f': '{0[0]:} {0[1]:} {0[2]:} {0[3]:}'.format
        },
        col_space={
            'nos_id': 12,
            't': 16,
            'v': 7,
            's': 7,
            'f': 9
        },
        header=False,
        index=False
    )
)

print(out_string)